In [1]:
import pandas as pd
import tensorflow as tf

In [3]:
columns_name = ['SepalLength','SepalWidth','PetalLength', 'PetalWidth', 'Species']
data_train = pd.read_csv('/content/iris_data/iris_training.csv',names=columns_name,header=0)
data_test = pd.read_csv('/content/iris_data/iris_test.csv',names=columns_name,header=0)

In [4]:
train_x, train_y = data_train.loc[:,'SepalLength':'PetalWidth'], data_train.loc[:,'Species']
test_x, test_y = data_test.loc[:,'SepalLength':'PetalWidth'], data_test['Species']
#label encode
mapping = {'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2}
train_y = train_y.map(mapping)
test_y = test_y.map(mapping)

In [5]:
my_feature_columns = []
for key in train_x.keys():
  print(key)
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

SepalLength
SepalWidth
PetalLength
PetalWidth
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [27]:
classifier = tf.estimator.DNNClassifier(# 這個模型接受哪些輸入的特徵
                     feature_columns=my_feature_columns,  
                     # 包含兩個隱藏層，每個隱藏層包含10個神經元.
                     hidden_units=[1000,450],
                     # 最終結果要分成的幾類
                     n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpi2hp0hyw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
def train_func(train_x,train_y):
  dataset=tf.data.Dataset.from_tensor_slices((dict(train_x),train_y))
  dataset = dataset.shuffle(1000).repeat().batch(100)
  return dataset

In [29]:
classifier.train(input_fn=lambda:train_func(train_x,train_y),steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpi2hp0hyw/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1149629, step = 0
INFO:tensorflow:global_step/sec: 82.7863
INFO:tensorflow:loss = 0.7222641, step = 100 (1.211 sec)
INFO:tensorflow:global_step/sec: 85.7575
INFO:tensorflow:loss = 0.5979551, step = 200 (1.165 sec)
INFO:tensorflow:global_step/sec: 88.6064
INFO:tensorflow:loss = 0.503668, step = 300 (1.129 sec)
INFO:tensorflow:global_step/sec: 88.8255
INFO:tensorflow:loss = 0.43757874, step = 400 (1.129 sec)
INFO:tensorflow:global_step/sec: 89.9805
INFO:tensorflow:loss = 0.3912561, step = 500 (1.113 sec)
INFO:t

In [30]:
def eval_input_fn(features, labels, batch_size):
  features=dict(features)
  if labels is None:
    # No labels, use only features.
    inputs = features
  else:
    inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    print(dataset)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
  return dataset

In [31]:
predict_arr = []
predictions = classifier.predict(
input_fn = lambda:eval_input_fn(test_x,labels=test_y,batch_size=100))

for predict in predictions:
  predict_arr.append(predict['probabilities'].argmax())
print(predict_arr)
result = predict_arr == test_y
result1 = [w for w in result if w == True]
print("準確率為 %s"%str((len(result1)/len(result))))

<TensorSliceDataset element_spec=({'SepalLength': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SepalWidth': TensorSpec(shape=(), dtype=tf.float64, name=None), 'PetalLength': TensorSpec(shape=(), dtype=tf.float64, name=None), 'PetalWidth': TensorSpec(shape=(), dtype=tf.float64, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpi2hp0hyw/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
準確率為 0.9818181818181818
